<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers



# load train and test data
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

# preprocess data

train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels_original, num_classes=10) 
test_labels = to_categorical(test_labels_original, num_classes=10)

# set up layers
model = keras.Sequential([
    keras.layers.Dense(10, input_dim=784, activation=tf.nn.softmax)
])

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# compile the model
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

# model.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

# train the model
epochs = 20
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(test_images, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 54us/step - loss: 0.4178 - acc: 0.8855 - val_loss: 0.3102 - val_acc: 0.9128
Epoch 2/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.3128 - acc: 0.9122 - val_loss: 0.2940 - val_acc: 0.9189
Epoch 3/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.2965 - acc: 0.9165 - val_loss: 0.2803 - val_acc: 0.9215
Epoch 4/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.2874 - acc: 0.9198 - val_loss: 0.2759 - val_acc: 0.9225
Epoch 5/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.2817 - acc: 0.9216 - val_loss: 0.2761 - val_acc: 0.9214
Epoch 6/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.2772 - acc: 0.9223 - val_loss: 0.2728 - val_acc: 0.9236
Epoch 7/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.2733 - acc: 0.9236 - val_loss: 0.2739 - val_acc